In [21]:
from dotenv import load_dotenv
import openai
import os
from openai import AzureOpenAI
import json 

load_dotenv()
SWEDEN_AZURE_OPENAI_ENDPOINT=os.getenv("SWEDEN_AZURE_OPENAI_ENDPOINT")
SWEDEN_AZURE_OPENAI_API_KEY=os.getenv("SWEDEN_AZURE_OPENAI_API_KEY")
SWEDEN_AZURE_OPENAI_API_VERSION="2024-08-01-preview"
SWEDEN_OPENAI_GPT4_DEPLOYMENT_NAME=os.getenv("SWEDEN_OPENAI_GPT4_DEPLOYMENT_NAME")
SWEDEN_OPENAI_GPT4o_DEPLOYMENT_NAME = os.getenv("SWEDEN_OPENAI_GPT4o_DEPLOYMENT_NAME")

client = openai.AzureOpenAI(
        azure_endpoint=SWEDEN_AZURE_OPENAI_ENDPOINT,
        api_key=SWEDEN_AZURE_OPENAI_API_KEY,
        api_version=SWEDEN_AZURE_OPENAI_API_VERSION
)

In [14]:
weather_function =     {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Determine weather in my location",
            "strict": True, # setting strict to true
            "parameters": {
              "type": "object",
              "properties": {
                "location": {
                  "type": "string",
                  "description": "The city and state e.g. San Francisco, CA"
                },
                "unit": {
                  "type": "string",
                  "enum": [
                    "c",
                    "f"
                  ]
                }
              },
              "required": [
                "location",
                "unit"
              ],
              "additionalProperties": False
            }
          },
    }

def get_response(user_input):
    response = client.chat.completions.create(
        model=SWEDEN_OPENAI_GPT4_DEPLOYMENT_NAME,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": user_input
            }
        ],
        tools=[weather_function]
    )

    return response.choices[0].message.tool_calls

In [15]:
def print_tool_call(user_input, tool_call):
    args = tool_call[0].function.arguments
    print(f"Input: {user_input}")
    print("Weather arguments:")
    for key, value in json.loads(args).items():
        print(f"{key}: '{value}'")
    print("\n\n")

In [16]:
input = "What's the weather in san francisco?"

result = get_response(input)

print_tool_call(input, result)

Input: What's the weather in san francisco?
Weather arguments:
location: 'San Francisco, CA'
unit: 'f'





#### JSON mode

In [22]:
math_tutor_prompt = '''
    You are a helpful math tutor. You will be provided with a math problem,
    and your goal will be to output a step by step solution, along with a final answer.
    For each step, just provide the output as an equation use the explanation field to detail the reasoning.
'''

In [23]:
import json
from pydantic import BaseModel

class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

In [24]:
def get_math_solution(question: str):
    completion = client.beta.chat.completions.parse(
        model=SWEDEN_OPENAI_GPT4o_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": math_tutor_prompt},
            {"role": "user", "content": question},
        ],
        response_format=MathReasoning,
    )

    return completion.choices[0].message

In [25]:
# Testing with an example question
question = "how can I solve 8x + 7 = -23"
result = get_math_solution(question)
print(result.parsed)

steps=[Step(explanation='First, we need to isolate the term with the variable on one side of the equation by removing the constant on the same side. We do this by subtracting 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplifying both sides, the +7 and -7 on the left cancel out, leaving just 8x. On the right side, we get -23 - 7, which equals -30.', output='8x = -30'), Step(explanation='Now, to solve for x, we need to get x by itself. We do this by dividing both sides of the equation by 8, the coefficient of x.', output='x = -30 / 8'), Step(explanation='Simplifying the right side by performing the division gives us -30 divided by 8, which simplifies to -3.75.', output='x = -3.75')] final_answer='x = -3.75'


In [26]:
result.parsed # supports autocompletion!

MathReasoning(steps=[Step(explanation='First, we need to isolate the term with the variable on one side of the equation by removing the constant on the same side. We do this by subtracting 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplifying both sides, the +7 and -7 on the left cancel out, leaving just 8x. On the right side, we get -23 - 7, which equals -30.', output='8x = -30'), Step(explanation='Now, to solve for x, we need to get x by itself. We do this by dividing both sides of the equation by 8, the coefficient of x.', output='x = -30 / 8'), Step(explanation='Simplifying the right side by performing the division gives us -30 divided by 8, which simplifies to -3.75.', output='x = -3.75')], final_answer='x = -3.75')

In [27]:
from IPython.display import Math, display

def print_math_response(response):
    result = json.loads(response)
    steps = result['steps']
    final_answer = result['final_answer']
    for i in range(len(steps)):
        print(f"Step {i+1}: {steps[i]['explanation']}\n")
        display(Math(steps[i]['output']))
        print("\n")

    print("Final answer:\n\n")
    display(Math(final_answer))

print_math_response(result.content)

Step 1: First, we need to isolate the term with the variable on one side of the equation by removing the constant on the same side. We do this by subtracting 7 from both sides of the equation.



<IPython.core.display.Math object>



Step 2: Simplifying both sides, the +7 and -7 on the left cancel out, leaving just 8x. On the right side, we get -23 - 7, which equals -30.



<IPython.core.display.Math object>



Step 3: Now, to solve for x, we need to get x by itself. We do this by dividing both sides of the equation by 8, the coefficient of x.



<IPython.core.display.Math object>



Step 4: Simplifying the right side by performing the division gives us -30 divided by 8, which simplifies to -3.75.



<IPython.core.display.Math object>



Final answer:




<IPython.core.display.Math object>

In [29]:
refusal_question = "What is the weather?"
refusal_result = get_math_solution(refusal_question)
print(refusal_result.refusal)

None
